In [ ]:
!nvidia-smi -L

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import os
import time
import torch
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

data_path = '../input/newsclass01'
output_path = '.'
FOLDER = f'{data_path}/images/images'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
df = pd.read_csv(f"{data_path}/train.csv")
images_filenames = []
images_labels = []
for index, row in df.iterrows():
    row_images = row['images']
    if isinstance(row_images, str):
        row_filenames = row_images.split(',')
        images_filenames += row_filenames
        images_labels += [row['source']] * len(row_filenames)
# images_filenames = images_filenames[0::50]
# images_labels = images_labels[0::50]
len(images_filenames)

In [ ]:
class ImageClassifierDataSet(Dataset):
    def __init__(self, transform, directory, filenames, labels):
        self.transform = transform
        self.directory = directory
        self.filenames = filenames
        self.labels = labels

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        return self.transform(Image.open(os.path.join(self.directory, self.filenames[index])).convert("RGB")), self.labels[index]

In [ ]:
def split_nth(splitee, nth):
    bigger_part = []
    for index in range(nth - 1):
        bigger_part += splitee[index + 1::nth]
    return splitee[0::nth], bigger_part

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_filenames, train_filenames = split_nth(images_filenames, 5)
val_labels, train_labels = split_nth(images_labels, 5)

trainset = ImageClassifierDataSet(transform, FOLDER, train_filenames, train_labels)
trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

valset = ImageClassifierDataSet(transform, FOLDER, val_filenames, val_labels)
valloader = DataLoader(valset, batch_size=4, shuffle=False, num_workers=2)

fullset = ImageClassifierDataSet(transform, FOLDER, train_filenames + val_filenames, train_labels + val_labels)
fullloader = DataLoader(fullset, batch_size=4, shuffle=False, num_workers=2)

In [ ]:
def epoch_phase(model, dataloader, criterion, optimizer, scheduler=None):
    phase_start = time.time()
    if scheduler is None:
        model.eval()
    else:
        model.train()
    running_loss = 0.0
    inputs_size = 0
    for index, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(scheduler is not None):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            if scheduler is not None:
                loss.backward()
                optimizer.step()
        inputs_size += inputs.size(0)
        running_loss += loss.item() * inputs.size(0)
        phase_time_left = time.time() - phase_start
        print(f'\r{"Val" if scheduler is None else "Train"} {index}/{len(dataloader)} for {int(phase_time_left)}/{int(phase_time_left / (index + 1) * len(dataloader))}s loss: {running_loss / inputs_size}', end='')
    if scheduler is not None:
        scheduler.step()

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f'\r{"Val" if scheduler is None else "Train"} Loss: {epoch_loss} for {time.time() - phase_start}s')
    return epoch_loss

In [ ]:
def train(model, trainloader, valloader, criterion, optimizer, scheduler, num_epochs=1):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        epoch_phase(model, trainloader, criterion, optimizer, scheduler)
        val_loss = epoch_phase(model, valloader, criterion, optimizer)
        path = output_path + f'/image_model_{int(time.time())}_{val_loss}.pth'
        torch.save(model, path)
    return model

In [ ]:
def train_fully(model, fullloader, criterion, optimizer, scheduler, num_epochs=1):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        loss = epoch_phase(model, fullloader, criterion, optimizer, scheduler)
        path = output_path + f'/image_model_{int(time.time())}_{loss}.pth'
        torch.save(model, path)
    return model

In [ ]:
def prepare_and_fully_train(model, fullloader, num_epochs):
    model = model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.00025, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
    return train_fully(model, fullloader, criterion, optimizer, scheduler, num_epochs)

In [ ]:
def prepare_and_train(trainloader, valloader, model, num_epochs):
    model = model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.00025, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
    return train(model, trainloader, valloader, criterion, optimizer, scheduler, num_epochs)

In [ ]:
model = torch.load('../input/news-image-classification-model-1617054602/image_model_1617054602.pth')
# prepare_and_train(trainloader, valloader, model, 4)

In [ ]:
prepare_and_fully_train(model, fullloader, num_epochs=5)

In [ ]:
import gc
gc.collect()
torch.cuda.memory_allocated()